In [1]:
# 1. Install datasets if you haven't already
!pip install datasets

# 2. Import the load_dataset function
from datasets import load_dataset

# 3. Load the dataset
dataset = load_dataset("SKNahin/bengali-transliteration-data")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/300 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/333k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5006 [00:00<?, ? examples/s]

In [2]:
# 4. Inspect the dataset
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['bn', 'rm'],
        num_rows: 5006
    })
})


In [3]:
from datasets import load_dataset

# Suppose you've already loaded the dataset into "dataset"
# dataset = load_dataset("SKNahin/bengali-transliteration-data")

# Perform an 80/20 train-validation split
split_dataset = dataset["train"].train_test_split(
    test_size=0.2,  # 20% for validation
    seed=42         # for reproducibility
)

# Extract the new train and validation subsets
train_ds = split_dataset["train"]
val_ds = split_dataset["test"]

print("Training set size:", len(train_ds))
print("Validation set size:", len(val_ds))

Training set size: 4004
Validation set size: 1002


In [4]:
train_ds = train_ds.rename_column("rm", "source")
train_ds = train_ds.rename_column("bn", "target")

val_ds = val_ds.rename_column("rm", "source")
val_ds = val_ds.rename_column("bn", "target")


In [5]:
def filter_examples(example, min_chars, max_chars):
    source_len = len(example["source"].strip())
    target_len = len(example["target"].strip())
    return (min_chars <= source_len <= max_chars) and (min_chars <= target_len <= max_chars)

train_ds = train_ds.filter(lambda ex: filter_examples(ex, 3, 40))
val_ds   = val_ds.filter(lambda ex: filter_examples(ex, 3, 40))

Filter:   0%|          | 0/4004 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1002 [00:00<?, ? examples/s]

In [6]:
pip install datasets transformers


In [7]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
import unicodedata

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [8]:
# Specify the pre-trained model name
model_name = "google/mt5-base"  # You can change to "google/mt5-base" if desired

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [9]:
# Define maximum sequence lengths based on filtering
max_source_length = 40   # Maximum characters in Banglish text
max_target_length = 40   # Maximum characters in Bangla text

In [10]:
def preprocess_function(examples):
    """
    Tokenizes the source and target texts.

    Args:
        examples (dict): A batch of examples containing 'source' and 'target' keys.

    Returns:
        dict: Tokenized inputs and labels.
    """
    # Tokenize the source (Banglish) texts
    model_inputs = tokenizer(
        examples["source"],
        max_length=max_source_length,
        padding='max_length',       # Pads to max_length
        truncation=True,            # Truncates to max_length
    )

    # Tokenize the target (Bangla) texts
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["target"],
            max_length=max_target_length,
            padding='max_length',
            truncation=True,
        )

    # Assign the tokenized target texts as labels
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


In [11]:
# Apply the preprocessing/tokenization to the training dataset
tokenized_train_ds = train_ds.map(
    preprocess_function,
    batched=True,                # Process in batches for efficiency
    remove_columns=["source", "target"]  # Remove original columns
)

# Apply the preprocessing/tokenization to the validation dataset
tokenized_val_ds = val_ds.map(
    preprocess_function,
    batched=True,
    remove_columns=["source", "target"]
)


Map:   0%|          | 0/2784 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/679 [00:00<?, ? examples/s]

In [12]:
def preprocess_function(examples):
    inputs = tokenizer(examples["source"], max_length=max_source_length, truncation=True, padding="max_length")
    targets = tokenizer(examples["target"], max_length=max_target_length, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs



In [13]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-base")


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [14]:
import os

output_dir = "./mt5_banglish_to_bangla"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [15]:
try:
    test_path = "./mt5_banglish_to_bangla/test_write.txt"
    with open(test_path, "w") as f:
        f.write("Test write permissions")
    print("Write permissions are enabled.")
    os.remove(test_path)  # Clean up the test file
except Exception as e:
    print("Write permissions are not enabled:", e)


Write permissions are enabled.


In [16]:
from transformers import Seq2SeqTrainingArguments


training_args = Seq2SeqTrainingArguments(
    output_dir="./mt5_banglish_to_bangla",
    run_name="banglish_to_bangla_experiment",  # Custom run name
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_steps=500,
    eval_steps=500,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=2,
    predict_with_generate=True
)


In [17]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_val_ds,
    data_collator=data_collator,  # Handles padding and batching
)


In [18]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [19]:
import torch
print("GPU available:", torch.cuda.is_available())


GPU available: True


In [ ]:
trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: